In [0]:
import pandas as pd
import os
from pyspark.sql.functions import *

In [0]:
def child_struct(nested_df):
    # Creating python list to store dataframe metadata
    list_schema = [((), nested_df)]
    # Creating empty python list for final flattern columns
    flat_columns = []

    while len(list_schema) > 0:
      # Removing latest or recently added item (dataframe schema) and returning into df variable  
          parents, df = list_schema.pop()
          flat_cols = [  col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"   ]
      
          struct_cols = [  c[0]   for c in df.dtypes if c[1][:6] == "struct"   ]
      
          flat_columns.extend(flat_cols)
          #Reading  nested columns and appending into stack list
          for i in struct_cols:
                projected_df = df.select(i + ".*")
                list_schema.append((parents + (i,), projected_df))
    return nested_df.select(flat_columns)

In [0]:
def master_array(df):
    array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    while len(array_cols)>0:
        for c in array_cols:
            df = df.withColumn(c,explode_outer(c))
        df = child_struct(df)
        array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    return df

## Doing transformation for all json file


In [0]:
dir = "/mnt/youtube_da2/transformed-data/youtube/cleaned_json_ver"
def file_exists(dir):
  try:
    dbutils.fs.ls(dir)
  except:
    return False  
  return True

In [0]:
%fs rm -r "/mnt/youtube_da2/transformed-data/youtube/cleaned_json_ver"

res0: Boolean = true

In [0]:
import os

if os.path.exists("/mnt/table_name"):
    os.remove("/mnt/table_name")

In [0]:
table_name = []

for i in dbutils.fs.ls('/mnt/youtube_da2/raw-data-json/ raw_statistics_reference_data/archive/'):
    table_name.append(i.name.split('/')[0].split('.')[0])


In [0]:
table_name

['CA_category_id',
 'DE_category_id',
 'GB_category_id',
 'JP_category_id',
 'KR_category_id',
 'US_category_id']

In [0]:
spark.conf.set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")

spark.conf.set("parquet.enable.summary-metadata", "false")

spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

In [0]:
output_path = '/mnt/youtube_da2/transformed-data/youtube/cleaned_json_ver'

for i in table_name:
    path = '/mnt/youtube_da2/raw-data-json/ raw_statistics_reference_data/archive/' + i +'.json'
    df = spark.read.option("multiline","true").json(path)
    df_remove = df.drop('etag','kind')

    df_output = master_array(df_remove)
    df_output.write.format('parquet').mode("overwrite").save(output_path)


In [0]:
# Get the list of files in the specified directory
file_list = dbutils.fs.ls(output_path)

# Filter files that start with "part-00000"
filtered_files = [file for file in file_list if file.name.startswith("part-00000")]

# Get the path of the first matching file
source_path = filtered_files[0].path

# Move the file to the new destination
destination_path = output_path + "/data.parquet"
dbutils.fs.mv(source_path, destination_path)

True

In [0]:
df_output = df_output.withColumn("items_id", df_output['items_id'].cast('int'))

In [0]:
display(df_output)

items_etag,items_id,items_kind,items_snippet_assignable,items_snippet_channelId,items_snippet_title
"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ""",1,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation
"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA""",2,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles
"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg""",10,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Music
"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI""",15,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals
"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM1XVQ9zbGec""",17,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Sports
"""m2yskBQFythfE4irbTIeOgYYfBU/FJwVpGCVZ1yiJrqZbpqe68Sy_OE""",18,youtube#videoCategory,false,UCBR8-60-B28hp2BmDPdntcQ,Short Movies
"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf_DkLN8CouA""",19,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Travel & Events
"""m2yskBQFythfE4irbTIeOgYYfBU/WmA0qYEfjWsAoyJFSw2zinhn2wM""",20,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,Gaming
"""m2yskBQFythfE4irbTIeOgYYfBU/EapFaGYG7K0StIXVf8aba249tdM""",21,youtube#videoCategory,false,UCBR8-60-B28hp2BmDPdntcQ,Videoblogging
"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZbNIytUQDRo""",22,youtube#videoCategory,true,UCBR8-60-B28hp2BmDPdntcQ,People & Blogs


In [0]:
df_count = df_output.count()
print(df_count)

32
